In [3]:
!pip install gdown
!gdown --id 1NM4uSjSg0e2LPBT_aC0v1XaRRh932jxJ
!gdown --id 19IDNSFnAh-JiVx05BSJJRmU-eTdzlG1T
!gdown --id 1ol6w0-OaxEe74GBVRP2edb0iBsOViV8i # film_adaptations
!gdown --id 1-08GhVIT-LcsDodcVaVep7LJeaCCbGfx # description_emb_books.csv
!gdown --id 1-MZoIhuMe6j5q48HDCe7AG9MTL5-mKkM # description_emb_films.csv
!gdown --id 10VHCp9-Y-blrnOjncEjF2V8yLI_Od5TR # TF IDF features books
!gdown --id 1SRoNRVl-mrnpRV0sMTC-qZxLOKa1gF4a # TF IDF features films

/opt/conda/lib/python3.7/site-packages/gdown/cli.py:125: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1NM4uSjSg0e2LPBT_aC0v1XaRRh932jxJ
To: /kaggle/working/books.csv
100%|███████████████████████████████████████| 69.0M/69.0M [00:00<00:00, 240MB/s]
/opt/conda/lib/python3.7/site-packages/gdown/cli.py:125: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=19IDNSFnAh-JiVx05BSJJRmU-eTdzlG1T
To: /kaggle/working/films.csv
100%|███████████████████████████████████████| 28.6M/28.6M [00:00<00:00, 197MB/s]
/opt/conda/lib/python3.7/site-packages/gdown/cli.py:125: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't ne

In [4]:
!pip install catboost
!pip install optuna

In [5]:
import numpy as np
import pandas as pd

import optuna

import scipy
from scipy.sparse import csr_matrix
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score

import warnings
warnings.filterwarnings('always')
import gc

### Сбор датасета

#### Подготовка таргета

In [6]:
film_adaptations = pd.read_csv('film_adaptations.csv')
film_adaptations['target'] = 1

films = pd.read_csv('films.csv')
books = pd.read_csv('books.csv')

In [7]:
books = books[~books.isna().any(axis=1)].reset_index(drop=True)
films = films[~films.isna().any(axis=1)].reset_index(drop=True)

In [8]:
films_ = films[['id', 'title', 'year']].rename({'id': 'film_id', 'title': 'film_title', 'year': 'film_year'}, axis=1)
books_ = books[['id', 'title', 'authors']].rename({'id': 'book_id', 'title': 'book_title', 'authors': 'book_authors'}, axis=1)

In [9]:
neg_sampler_train_valid = pd.concat([
    books_.sample(100000, replace=True, random_state=42).reset_index(drop=True),
    films_.sample(100000, replace=True, random_state=42).reset_index(drop=True)
], axis=1)

neg_sampler_train_valid['target'] = 0

train_valid = pd.concat([film_adaptations, neg_sampler_train_valid])
del train_valid['film_rating']
train_valid = train_valid.drop_duplicates(subset=['book_title', 'book_id', 'book_authors', 'film_title', 'film_id', 'film_year'])
train_valid = train_valid.sample(frac=1.0).reset_index(drop=True)

In [10]:
test = pd.concat([
    books_.sample(5, random_state=42).sample(100000, replace=True, random_state=42).reset_index(drop=True),
    films_.sample(100000, replace=True, random_state=42).reset_index(drop=True)
], axis=1)

test = test.drop_duplicates(subset=['book_title', 'book_id', 'book_authors', 'film_title', 'film_id', 'film_year'])

In [11]:
# data = data[['book_id', 'film_id', 'target']]
del books, films, films_, books_, neg_sampler_train_valid, film_adaptations
gc.collect()

95

In [12]:
# data.to_csv('film_adapations_final_df.csv', index=False)

#### Добавление фичей

In [18]:
description_emb_tf_idf_books = pd.read_csv('tf_idf_books.csv', index_col=0, dtype=np.float32)
description_emb_tf_idf_films = pd.read_csv('tf_idf_films.csv', index_col=0, dtype=np.float32)

# description_emb_tf_idf_books = csr_matrix(description_emb_tf_idf_books.astype(pd.SparseDtype("float64",0)).sparse.to_coo())
# description_emb_tf_idf_films = csr_matrix(description_emb_tf_idf_films.astype(pd.SparseDtype("float64",0)).sparse.to_coo())

description_emb_tf_idf_books = (
    description_emb_tf_idf_books
    .reset_index()
    .rename({'index': 'book_id'}, axis=1)
)

description_emb_tf_idf_films = (
    description_emb_tf_idf_films
    .reset_index()
    .rename({'index': 'film_id'}, axis=1)
)

In [20]:
description_emb_tf_idf_books.book_id = description_emb_tf_idf_books.book_id.astype(int)
description_emb_tf_idf_films.film_id = description_emb_tf_idf_films.film_id.astype(int)

In [21]:
train_valid

,book_title,book_id,book_authors,film_title,film_id,film_year,target
0,На войне как на войне,27571,Сергей Самаров,Про вола,12238,2014.0,0
1,Ключ темных,25286,Евгений Катрич,Update/Delete,5043,2018.0,0
2,Золотко,20852,Мария Мещерская,Офицерские жены,931,2015.0,0
3,Моя Рогатая Практика,10103,Алекс Найт,План побега 2,15833,2018.0,0
4,Куратор Истории,54411,Юрий Иванович,Злой дух Ямбуя,21117,1978.0,0
...,...,...,...,...,...,...,...
102670,Железное кольцо,1136,Дмитрий Мережковский,"Трое в лодке, не считая собаки",14051,1979.0,0
102671,Ксеноцид,35230,Орсон Скотт Кард,Любовь без правил,15434,2010.0,0
102672,Виндсорские кумушки. Комедия в пяти действиях ...,4398,Виссарион Белинский,Малыш Франкенштейн,19560,2018.0,0
102673,Авто Секс,7772,Карл Шорт,Кабачок «13 стульев»,1970,1966.0,0


In [22]:
print('count data before adding some features', train_valid.shape)
train_valid = train_valid.merge(description_emb_tf_idf_books, on=['book_id'], how='left')
test = test.merge(description_emb_tf_idf_books, on=['book_id'], how='left')
# del description_emb_tf_idf_books
gc.collect()
train_valid = train_valid.merge(description_emb_tf_idf_films, on=['film_id'], how='left')
test = test.merge(description_emb_tf_idf_films, on=['film_id'], how='left')
# del description_emb_tf_idf_films
gc.collect()
print('count data after adding some features', train_valid.shape)

count data before adding some features (102675, 7)
count data after adding some features (102675, 16305)


In [23]:
# description_emb_books = pd.read_csv('description_emb_books.csv')
# description_emb_films = pd.read_csv('description_emb_films.csv')

# escription_emb_books = description_emb_books.rename({'id': 'book_id', 'title': 'book_title', 'authors': 'book_authors'}, axis=1)
# description_emb_films = description_emb_films.rename({'id': 'film_id', 'title': 'film_title', 'year': 'film_year'}, axis=1)

# print('count data before adding some features', data.shape)
# data = data.merge(description_emb_books, on=['book_id', 'book_title', 'book_authors'])
# data = data.merge(description_emb_films, on=['film_id', 'film_title', 'film_year'])
# print('count data after adding some features', data.shape)

# Обучение модели

## бейзлайн

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(train_valid.drop('target', axis=1), train_valid['target'], test_size=0.1, random_state=0, stratify=data['target'])

In [ ]:
# cols_exclude_train = [
#     'book_title', 'book_id', 'book_authors',
#     'film_title', 'film_id', 'film_year', 'film_rating'
# ]

cols_exclude_train = [
    'book_id', 'film_id'
]

In [ ]:
model = CatBoostClassifier(iterations=200, verbose=0)
model.fit(X_train.drop(cols_exclude_train, axis=1), y_train)

In [ ]:
print("_________TRAIN_METRICS__________")
print(classification_report(y_train, model.predict(X_train.drop(cols_exclude_train, axis=1))))
print()
print("_________TEST_METRICS__________")
print(classification_report(y_valid, model.predict(X_valid.drop(cols_exclude_train, axis=1))))

In [40]:
# print("_________TRAIN_METRICS__________")
# print(classification_report(
#     y_train, 
#     (model.predict_proba(X_train.drop(cols_exclude_train, axis=1)) > 0.1)[:, 1].astype(int)
# ))
# print()
# print("_________TEST_METRICS__________")
# print(classification_report(
#     y_test, 
#     (model.predict_proba(X_test.drop(cols_exclude_train, axis=1)) > 0.1)[:, 1].astype(int)
# ))

## Optuna

In [23]:
def objective(trial):
    train_x, valid_x, train_y, valid_y = train_test_split(X_train.drop(cols_exclude_train, axis=1), y_train, test_size=0.2)

    param = {
        "objective": trial.suggest_categorical("objective", ["Logloss", "CrossEntropy"]),
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.01, 0.1),
        "depth": trial.suggest_int("depth", 1, 12),
        "boosting_type": trial.suggest_categorical("boosting_type", ["Ordered", "Plain"]),
        "bootstrap_type": trial.suggest_categorical(
            "bootstrap_type", ["Bayesian", "Bernoulli", "MVS"]
        ),
    }

    if param["bootstrap_type"] == "Bayesian":
        param["bagging_temperature"] = trial.suggest_float("bagging_temperature", 0, 10)
    elif param["bootstrap_type"] == "Bernoulli":
        param["subsample"] = trial.suggest_float("subsample", 0.1, 1)

    gbm = CatBoostClassifier(**param, verbose=1, iterations=100)

    gbm.fit(train_x, train_y, eval_set=[(valid_x, valid_y)], verbose=0, early_stopping_rounds=100)

    preds = gbm.predict(valid_x)
    pred_labels = np.rint(preds)
    accuracy = f1_score(valid_y, pred_labels)
    return accuracy

# study = optuna.create_study(direction="maximize")
# study.optimize(objective, n_trials=10, timeout=60)

# gbm = CatBoostClassifier(**study.best_params, verbose=1, iterations=100)
# gbm.fit(X_train.drop(cols_exclude_train, axis=1), y_train, verbose=0, early_stopping_rounds=100)

# print("_________TRAIN_METRICS__________")
# print(classification_report(
#     y_train, 
#     (gbm.predict_proba(X_train.drop(cols_exclude_train, axis=1)) > 0.1)[:, 1].astype(int)
# ))
# print()
# print("_________TEST_METRICS__________")
# print(classification_report(
#     y_test, 
#     (gbm.predict_proba(X_test.drop(cols_exclude_train, axis=1)) > 0.1)[:, 1].astype(int)
# ))

# Просмотр результатов

In [48]:
books_genres = books[['id', 'title', 'genres']].rename({'id': 'book_id', 'title': 'book_title', 'genres': 'book_genres'}, axis=1)
films_genres = films[['id', 'title', 'genres']].rename({'id': 'film_id', 'title': 'film_title', 'genres': 'film_genres'}, axis=1)

X_test_ = (
    X_test
    .copy()
    [['book_id', 'film_id']]
    .merge(books_genres, on=['book_id'])
    .merge(films_genres, on=['film_id'])
)
X_test_['pred'] = (model.predict_proba(X_test.drop(cols_exclude_train, axis=1)) > 0.001)[:, 1].astype(int)

In [49]:
X_test_[(X_test_.pred == 1) & (y_test == 0)].drop_duplicates().sample(30)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,book_id,film_id,book_title,book_genres,film_title,film_genres,pred
6667,5568,8728,Погоня,"Зарубежные любовные романы,Современные любовны...",Во власти золота,драмы,1
8743,15416,19680,Я #самая желанная #самая счастливая! Лучшая пр...,"Любовь и отношения,Семейная психология",В фокусе,"драмы, зарубежные, комедии",1
7920,15960,2952,Отказываюсь выбирать! Как использовать свои ин...,"Зарубежная психология,Мотивация,Саморазвитие /...",Про Любовь,комедии,1
7488,6020,139,"Маска, я тебя не знаю!",Современные детективы,Любовь и прочий зоопарк,"мелодрамы, комедии",1
9245,2450,12592,Можно попросить Нину?,"Классика фантастики,Научная фантастика,Социаль...",Золотой ключик,"семейное, русские, фэнтези",1
9081,47904,4360,Исполняющий желания,"Боевое фэнтези,Героическое фэнтези,Городское ф...",Воин.com,комедии,1
9764,23856,865,Рожденная в пламени ночи,"Любовно-фантастические романы,Остросюжетные лю...",Шери,"драмы, зарубежные, мелодрамы",1
1407,14800,22064,Несчастный случай,"Зарубежная фантастика,Космическая фантастика,Н...",Всадники,"исторические, русские, военные",1
2664,8440,13672,"Сердце не обманет, сердце не предаст",Современные детективы,"Сердце не обманет, сердце не предаст","русские, детективы",1
7627,11328,21808,Магический турнир. Начало,"Детективное фэнтези,Книги про волшебников,Любо...",Битва титанов,"боевики, фэнтези, приключения",1
